In [ ]:
def remove_number(sentence):
    # Находим первое вхождение пробела
    space_index = sentence.find(' ')
    if space_index != -1:
        # Удаляем все символы перед первым пробелом
        sentence = sentence[space_index+1:]
    return sentence

# Открываем файл для чтения
with open('data.txt', 'r') as file:
    # Читаем все строки из файла
    lines = file.readlines()

# Создаем список для хранения обработанных предложений
processed_sentences = []

# Обрабатываем каждую строку из файла
for line in lines:
    # Удаляем числа из начала строки
    processed_sentence = remove_number(line.strip())
    # Добавляем обработанное предложение в список
    processed_sentences.append(processed_sentence)

# Открываем файл для записи
with open('data2.txt', 'w') as file:
    # Записываем обработанные предложения в файл
    for sentence in processed_sentences:
        file.write(sentence + '\n')


In [ ]:
import re

# Функция для извлечения слов из {}
def extract_keywords(sentence):
    # Используем регулярное выражение для поиска слов в {}
    matches = re.findall(r'{(.*?)}', sentence)
    return matches

# Открываем файл data2.txt для чтения
with open('data2.txt', 'r') as file:
    # Читаем все строки из файла
    lines = file.readlines()

# Создаем список для хранения ключевых слов
keywords_list = []
# Обрабатываем каждую строку из файла
for line in lines:
    # Извлекаем ключевые слова из строки
    keywords = extract_keywords(line)
    # Добавляем список ключевых слов в общий список
    keywords_list.append(keywords)


# Открываем файл keywords_oracle.txt для записи
with open('keywords_oracle.txt', 'w') as file:
    # Записываем ключевые слова в файл
    for keywords in keywords_list:
        # Удаляем лишние символы и записываем ключевые слова через запятую
        cleaned_keywords = [re.sub(r'[^\w\s]', '', keyword) for keyword in keywords]
        file.write(','.join(cleaned_keywords) + '\n')


In [2]:
# Открываем файл keywords_oracle.txt для чтения
with open('keywords_oracle.txt', 'r') as file:
    # Читаем все строки из файла
    lines = file.readlines()

# Инициализируем переменную для подсчета слов
total_words = 0

# Обрабатываем каждую строку из файла
for line in lines:
    # Разделяем строку на слова по запятой и считаем количество слов
    total_words += len(line.split(','))

# Выводим общее количество слов
print("Общее количество слов:", total_words)

Общее количество слов: 334


In [ ]:
# Открываем файл keywords_oracle.txt для чтения
with open('keywords_oracle.txt', 'r') as file:
    # Читаем все строки из файла
    lines = file.readlines()

# Создаем список для хранения слов в каждой строке
word_lists = []

# Обрабатываем каждую строку из файла
for line in lines:
    # Разделяем строку на слова по запятой и добавляем их в список
    words = line.strip().split(',')
    # Добавляем список слов в общий список
    word_lists.append(words)

# Выводим слова в каждой строке
#for i, words in enumerate(word_lists, start=1):
#    print(f"Слова в строке {i}: {words}")
#

In [3]:
%%capture
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers
!pip install keybert ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

In [4]:

# Стоп-слова в виде списка
stop_words_list = ["locate","identify", "spot", "find", "discover", "pinpoint","Locate","Identify","Spot","Find","Discover", "Pinpoint"]


In [10]:
from keybert import KeyBERT
import time

# Инициализация модели KeyBERT
kw_model = KeyBERT()

# Открытие файла data3.txt для чтения
with open("data3.txt", "r") as file:
    lines = file.readlines()

# Открытие файла keywords_BERT.txt для записи
with open("keywords_BERT.txt", "w") as file:
    with open("time_BERT.txt", "w") as time_file:
      # Проход по строкам из файла data3.txt
      for line in lines:
          # Извлечение ключевых слов из текущей строки
          # Проводим подсчет времени на обработку строки из data3.txt
          start_time = time.time()
          keywords = kw_model.extract_keywords(line, keyphrase_ngram_range=(1, 1), stop_words = stop_words_list)
          end_time = time.time()
          # Время в миллисекундах
          execution_time = (end_time - start_time) * 1000
          # Формирование строки с ключевыми словами, разделенными запятыми

          filtered_keywords = [(word, score) for word, score in keywords if score > 0.35]

          keyword_string = ','.join([word for word, score in filtered_keywords])
          # Запись строки с ключевыми словами в файл keywords_BERT.txt
          file.write(keyword_string + '\n')
          # Запись времени выполнения в файл time_BERT.txt
          time_file.write(f"{execution_time:.3f}\n")


In [11]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [12]:
from transformers import AutoTokenizer, pipeline

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [44]:

prompt = """
I need you to identify the objects (or object) that are the main ones in this sentence.
*"Discover the mountain towering above the lake and the boat sailing on its water."
You must to only return the main objects(or object) in one line separated by commas and say nothing else. The line must started with '_' and end with '_'
"""

response = generator(prompt)


text = response[0]["generated_text"][len(prompt):]
print(text)


text = text.strip('_')

# Разбиваем строку на список слов по запятым
words = text.split(', ')

print(words)


'_mountain, boat_'
["'_mountain", "boat_'"]


In [47]:
prompt = """
I need you to identify the objects (or object) that are the main ones in this sentence.
*"""

line = "Identify the car driving on the road."

directive = """
You must only return the main objects(or object) in one line separated by commas and say nothing else. The line must started with '_' and end with '_'
"""

'\nI need you to identify the objects (or object) that are the main ones in this sentence.\n*"Identify the lamp on the nightstand and the book beside it."\n\nYou must only return the main objects(or object) in one line separated by commas and say nothing else. The line must started with \'_\' and end with \'_\'\n\n_Lamp, Book_'

In [48]:
import re
prompt = """
I need you to identify the objects (or object) that are the main ones in this sentence.
*"""
directive = """
You must only return the main objects(or object) in one line separated by commas and say nothing else. The line must started with '_' and end with '_'
"""
with open("data3.txt", "r") as file:
    lines = file.readlines()
with open("keywords_MISTRAL.txt", "w") as file:
    with open("time_MISTRAL.txt", "w") as time_file:
      # Проход по строкам из файла data3.txt
      for line in lines:
          # Извлечение ключевых слов из текущей строки
          # Проводим подсчет времени на обработку строки из data3.txt
          start_time = time.time()
          response = generator(prompt + line + directive)
          end_time = time.time()
          # Время в миллисекундах
          execution_time = (end_time - start_time) * 1000
          # Удаляем символы "_" из строки
          text = response[0]["generated_text"][len(prompt + line + directive):].strip('_')
          # Разбиваем строку по запятым и пробелам
          words = re.split(r',\s*|\s+', text)
          # Формирование строки с ключевыми словами, разделенными запятыми
          keyword_string = ','.join(words)
          # Запись строки с ключевыми словами в файл keywords_MISTRAL.txt
          file.write(keyword_string + '\n')
          # Запись времени выполнения в файл time_MISTRAL.txt
          time_file.write(f"{execution_time:.3f}\n")